<a href="https://colab.research.google.com/github/Ori4est/LAMA/blob/main/Copy_of_EOCR(CRAFT)_LAMA_Prods_TextRemoval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gitpython
import git

In [2]:
from google.colab import drive
drive.mount("/content/gdrive") # force_remount=True

Mounted at /content/gdrive


In [1]:
#@title ### LAMA-inpaint ###
#Run this cell to set everything up
print('\n> Cloning the repo')
!git clone https://github.com/Ori4est/LAMA.git

print('\n> Install dependencies')
!pip install wldhx.yadisk-direct
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival
!pip uninstall kornia -y
!pip install kornia --no-dependencies
!pip install kornia-rs
!pip install pytorch-lightning
!pip install hydra-core
!pip install webdataset
!pip install torch torchvision torchaudio torchtext
!pip install -r LAMA/requirements.txt --quiet
!pip install wget --quiet


print('\n> Changing the dir to:')
%cd /content/LAMA

#### link inconsistent!!! ####
#print('\n> Download the model')
#!curl -L $(yadisk-direct https://disk.yandex.ru/d/ouP6l8VJ0HpMZg) -o big-lama.zip
#!unzip big-lama.zip

print('>fixing opencv')
!pip uninstall opencv-python-headless -y --quiet
!pip install opencv-python-headless==4.1.2.30 --quiet
!pip install --upgrade opencv-python


print('\n> Init mask-drawing code')
import base64, os
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
import matplotlib.pyplot as plt
import numpy as np
import wget
from shutil import copyfile
import shutil



> Cloning the repo
fatal: destination path 'LAMA' already exists and is not an empty directory.

> Install dependencies
  Using cached osqp-0.6.7.post3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
Using cached osqp-0.6.7.post3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (297 kB)
Found existing installation: kornia 0.7.4
Uninstalling kornia-0.7.4:
  Successfully uninstalled kornia-0.7.4
  Using cached kornia-0.7.4-py2.py3-none-any.whl.metadata (18 kB)
Using cached kornia-0.7.4-py2.py3-none-any.whl (899 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.8/29.8 MB 99.9 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparin

In [2]:
# create symbolic links to model weights
# temp fix for the inconsistant big-lama download link
!ln -s "/content/gdrive/MyDrive/lama/big-lama" $(pwd)/big-lama

In [3]:
# fix subprocess error
!pip install opencv-python-headless
!pip install albumentations==0.5.2
!pip install pillow-avif-plugin Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 78.0 MB/s eta 0:00:00
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.4.20
    Uninstalling albumentations-1.4.20:
      Successfully uninstalled albumentations-1.4.20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 106.5 MB/s eta 0:00:00


In [4]:
#@title ### EASYOCR ###
!pip install git+https://github.com/Ori4est/EasyOCR.git

  Cloning https://github.com/Ori4est/EasyOCR.git to /tmp/pip-req-build-0bhzaaxu
  Running command git clone --filter=blob:none --quiet https://github.com/Ori4est/EasyOCR.git /tmp/pip-req-build-0bhzaaxu
  Resolved https://github.com/Ori4est/EasyOCR.git to commit 2e0132e05797c5d2b6b476ee3c912b11002b0586
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 42.4 MB/s eta 0:00:00
  Created wheel for easyocr: filename=easyocr-1.7.2-py3-none-any.whl size=77041447 sha256=032e435deb82fe9844f9cd179826bcdc46b1b0c5de5e498afccaf18626355c75
  Stored in directory: /tmp/pip-ephem-wheel-cache-d01y_nr6/wheels/cb/c9/58/c9aa735df0cff55769d042c667d35b079d8aa5d4f06b371a50
Successfully built easyocr


In [ ]:
def apply_brightness(image):
    """apply brightness image augmentation to image, and return augmented image"""
    brightness = ImageEnhance.Brightness(image)
    #grab random float between 0.5 and 1.5
    brightness_factor = np.random.uniform(0.5, 1.5)
    image_bright = brightness.enhance(brightness_factor)
    return image_bright


def add_sharpening(image):
    return image.filter(ImageFilter.UnsharpMask())


def add_contrast(image):
    contrast = ImageEnhance.Contrast(image)
    # random contrast factor between 0.5 and 2
    factor = np.random.uniform(0.5, 2)
    return contrast.enhance(factor)  # Contrast factor


def add_saturation(image):
    saturation = ImageEnhance.Color(image)
    factor = np.random.uniform(0.5, 2)
    return saturation.enhance(factor)  # Saturation factor

In [ ]:
#@title Text detection
import easyocr
import os
from glob import glob
from tqdm import tqdm
from PIL import Image
import pillow_avif
from matplotlib import pyplot as plt
from PIL import ImageDraw
import cv2
import numpy as np

# Note: English is compatible with all languages. Languages that share most of character (e.g. latin script) with each other are compatible.
reader_EF = easyocr.Reader(['en', 'fr']) # this needs to run only once to load the model into memory
#reader_CN = easyocr.Reader(['cn_sim', 'cn_tra'])
#reader_JP = easyocr.Reader(['ja', 'en'])
#reader_KR = easyocr.Reader(['ko', 'en'])
#reader_DE = easyocr.Reader(['en', 'de']) # German
#reader_IT = easyocr.Reader(['en', 'it']) # Italian
#reader_ES = easyocr.Reader(['en', 'es']) # Spanish
#reader_PT = easyocr.Reader(['en', 'pt']) # Portuguese

#code to load custom craft model (from )
import easyocr
from easyocr.detection import get_detector, get_textbox
import torch
import cv2

image_filename_list = glob(input_dir+'/**/**/*.*')
images_path = [os.path.join(input_dir, file_path) for file_path in image_filename_list]
print(f"Total number of images: {len(images_path)}")

for img_path, img_name in tqdm(zip(images_path, image_filename_list)):
    if img_name == img_path:
        img_name = os.path.basename(img_name)
    if 'avif' in img_name:
        img = Image.open(img_path)
        img_path = img_path.replace('avif', 'png')
        img.save(img_path)

    det_results = reader_EF.readtext(img_path)
    #output_img = os.path.join(output_dir, filename)
    print(det_results)
    img = cv2.imread(img_path)
    for (bbox, text, prob) in det_results:
        startPoint, endPoint = np.array(bbox[0], dtype=np.int64), np.array(bbox[2], dtype=np.int64)
        print(startPoint, endPoint)
        cv2.rectangle(img, startPoint, endPoint, (0, 255, 0), thickness=2)
        cv2.putText(img, text, (int(bbox[0][0])-10, int(bbox[0][1])+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),)
    '''
    handwritten_results = handwriting_reader.readtext(input_img)
    for (bbox, text, prob) in handwritten_results:
        startPoint, endPoint = np.array(bbox[0], dtype=np.int64), np.array(bbox[2], dtype=np.int64)
        print(startPoint, endPoint)
        cv2.rectangle(img, startPoint, endPoint, (0, 255, 0), thickness=2)
        cv2.putText(img, text, (int(bbox[0][0])-10, int(bbox[0][1])+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0),)
    '''
    plt.axis("on")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{img_name.split('.')[0]}")
    plt.show()

    #



In [ ]:
#@title Mask drawing
from functools import reduce
from operator import add

def drawMask(points, image):
    points = np.array(points)
    cv2.fillPoly(image, [points], color=(255, 255, 255))
    return image

for filename in os.listdir(input_dir):
    input_img = os.path.join(input_dir, filename)
    det_results = reader.readtext(input_img)
    #output_img = os.path.join(output_dir, filename)
    print(det_results)
    img = cv2.imread(input_img)
    row, col, channel = img.shape
    img_mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    radius = 0

    for (bbox, text, prob) in det_results:
        startPoint, endPoint = np.array(bbox[0], dtype=np.int64), np.array(bbox[2], dtype=np.int64)
        points = reduce(add, bbox)
        xy1 = (max(int(points[0]) - radius, 0), max(int(points[1]) - radius, 0))
        xy2 = (min(int(points[2]) + radius, col-1), max(int(points[3]) - radius, 0))
        xy3 = (min(int(points[4]) + radius, col-1), min(int(points[5]) + radius, row-1))
        xy4 = (max(int(points[6]) - radius, 0), min(int(points[7]) + radius, row-1))
        points = np.array([xy1, xy2, xy3, xy4])

        img_mask = drawMask(points, img_mask)

        #cv2.rectangle(img, startPoint, endPoint, (0, 255, 0), thickness=2)
        #cv2.putText(img, text, (int(bbox[0][0])-10, int(bbox[0][1])+10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0),)
    plt.axis("on")
    plt.imshow(cv2.cvtColor(img_mask, cv2.COLOR_BGR2RGB))
    plt.title(f"Masked {filename.split('.')[0]}")
    plt.show()





In [ ]:
#@title copy from utils.py & demo.py
# Inpaint masked area
import os, sys
sys.path.append('/content/LAMA')
%cd '/content/LAMA'
from glob import glob
from shutil import copyfile
import yaml
from tqdm import tqdm
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt
import pillow_avif
from functools import reduce
from operator import add
import torch.nn as nn
import torch
from torch.utils.data._utils.collate import default_collate

import easyocr
from easyocr.detection import get_detector, get_textbox
import cv2

#from trocr.src.main import TrocrPredictor

from omegaconf import OmegaConf
from saicinpainting.evaluation.utils import move_to_device
from saicinpainting.evaluation.refinement import refine_predict
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['VECLIB_MAXIMUM_THREADS'] = '1'
os.environ['NUMEXPR_NUM_THREADS'] = '1'

from saicinpainting.training.data.datasets import make_default_val_dataset, get_transforms
from saicinpainting.evaluation.data import InpaintingDataset as InpaintingEvaluationDataset #
from saicinpainting.training.trainers import load_checkpoint
from saicinpainting.utils import register_debug_signal_handlers

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Note: English is compatible with all languages. Languages that share most of character (e.g. latin script) with each other are compatible.
# this needs to run only once to load the model into memory
reader_EF = easyocr.Reader(['en', 'fr'])
#reader_CN = easyocr.Reader(['cn_sim', 'cn_tra'])
#reader_JP = easyocr.Reader(['ja', 'en'])
#reader_KR = easyocr.Reader(['ko', 'en'])
#reader_DE = easyocr.Reader(['en', 'de']) # German
#reader_IT = easyocr.Reader(['en', 'it']) # Italian
#reader_ES = easyocr.Reader(['en', 'es']) # Spanish
#reader_PT = easyocr.Reader(['en', 'pt']) # Portuguese


def load_image(fname, mode='RGB', return_orig=False):
    img = np.array(Image.open(fname).convert(mode))
    if img.ndim == 3:
        img = np.transpose(img, (2, 0, 1))
    out_img = img.astype('float32') / 255
    if return_orig:
        return out_img, img
    else:
        return out_img


def hconcat_resize_min(im_list, interpolation=cv2.INTER_LANCZOS4):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)


def drawMask(points, image, device='cuda'):
    points = np.array(points)
    cv2.fillPoly(image, [points], color=(255, 255, 255))
    return image


def drawDilateMask(points, image, kernel_size=5, device='cuda'):
    points = np.array(points)
    cv2.fillPoly(image, [points], color=(255, 255, 255))
    cv2.dilate(image, (kernel_size, kernel_size), iterations=6)
    return image


def convert_from_cv2_to_image(img: np.ndarray) -> Image:
    # return Image.fromarray(img)
    if img.shape[-1] == 4:
        return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGRA2RGBA))
    elif img.shape[-1] == 3:
        return Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    else:
        raise ValueError(f"Invalid number of channels: {img.shape[-1]}")


def convert_from_image_to_cv2(img: Image) -> np.ndarray:
    # return np.asarray(img)
    if img.mode == 'RGBA':
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGBA2BGRA)
    elif img.mode == 'RGB':
        return cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    else:
        raise ValueError(f"Invalid mode: {img.mode}")


def style_detect(img, boundary_pts, color_thres=40):
    # for human part ONLY
    bg_color = False
    row, col, channel = img.shape
    alpha_mask = np.zeros((row, col), dtype=np.uint8)

    xy1 = (max(int(boundary_pts[0]), 0), max(int(boundary_pts[1]), 0))
    xy2 = (min(int(boundary_pts[2]), col-1), max(int(boundary_pts[3]), 0))
    xy3 = (min(int(boundary_pts[4]), col-1), min(int(boundary_pts[5]), row-1))
    xy4 = (max(int(boundary_pts[6]), 0), min(int(boundary_pts[7]), row-1))
    boundary = np.array([xy1, xy2, xy3, xy4])
    # obtain boundary size info
    center, wh, orient = cv2.minAreaRect(boundary)

    cv2.fillPoly(alpha_mask, [boundary], color=(255,))
    # count transparent area
    trans_arr = np.where(alpha_mask == 0)[0]
    print(trans_arr.shape)

    # create binary
    cropped_region = cv2.cvtColor(img, cv2.COLOR_RGB2RGBA) # BGR2RGBA?
    cropped_region[:, :, 3] = alpha_mask
    #image_file = image_file.convert('LA') # converts to grayscale w/ alpha

    img_gray = Image.fromarray(cropped_region).convert('LA') #img_gray = cv2.cvtColor(cropped_region, cv2.COLOR_BGR2GRAY)
    img_gray = np.array(img_gray).astype(np.uint8) # gray, cv2.cvtColor no need
    thres, img_bin = cv2.threshold(img_gray[:, :, 0], 100, 255, cv2.THRESH_OTSU)

    #plt.axis("on")
    ##plt.imshow(cv2.cvtColor(cropped_region, cv2.COLOR_RGBA2BGRA))
    #tmp_ = img_bin.copy()
    #tmp_[img_gray[:, :, -1] == 0] = 200
    #plt.imshow(cv2.cvtColor(tmp_, cv2.COLOR_RGBA2BGRA))
    #plt.title(f"blending with alpha channel {img.shape} {img_bin.shape} {trans_arr.shape}")
    #plt.show()
    #del tmp_

    img_bin[img_gray[:, :, -1] == 0] = 2 # non-transparent area
    img_bin[img_bin == 255] = 1

    #arr = np.sort(img_bin.sum(axis=0)) # sum by row
    #one_sum = arr[len(arr) - 1]
    #zero_sum = img_bin.shape[0] - arr[0] - trans_arr.shape[0]
    one_sum = len(np.where(img_bin == 1)[0])
    zero_sum = row*col - one_sum - len(trans_arr)
    print(f"one_sum: {one_sum}, zero_sum: {zero_sum}, redundance {one_sum * 0.8}")
    if zero_sum < one_sum * 0.8:
        img_bin = (1 - img_bin)

    # get component colors
    var_bg = np.where(img_bin == 0)
    colors = cropped_region[:,:,:3][var_bg[0], var_bg[1]]
    background_color = np.mean(colors, axis=0)
    # to reduce the overall computations, this ver only support blue text background search
    #if abs(background_color[2] - 50) < 10 and abs(background_color[1]-110) < 15 and abs(background_color[0]-160) < 20:

    n_iter = 1
    kernel = 5 #max(int(wh[0] / 10), int(wh[1] / 10))
    print(f'kernel check it out {kernel}')
    bound_ext0 = cv2.dilate(alpha_mask, (kernel, kernel), iterations=n_iter)
    var_edge_0 = np.where((bound_ext0 == 255) & (alpha_mask == 0))
    edge_0_colors = img[:,:,:3][var_edge_0[0], var_edge_0[1]]
    edge_0_mean = np.mean(edge_0_colors, axis=0)
    dist_0 = np.sqrt(np.sum((edge_0_mean - background_color)**2))

    # color distance
    while dist_0 < color_thres and n_iter < 3:
        n_iter += 1
        bound_ext1 = cv2.dilate(alpha_mask, (kernel, kernel), iterations=n_iter)
        var_edge_0 = np.where((bound_ext1 == 255) & (bound_ext0 == 0))
        edge_0_colors = img[:,:,:3][var_edge_0[0], var_edge_0[1]]
        edge_0_mean = np.mean(edge_0_colors, axis=0)
        dist_0 = np.sqrt(np.sum((edge_0_mean - background_color)**2))
        bound_ext0 = bound_ext1 # renew
        print(f"find the edge {dist_0} {n_iter}")
        bg_color = True
    if n_iter * kernel > min(int(wh[0]), int(wh[1])):
        bg_color = False
        n_iter = 0

    edge_0_black = np.sqrt(np.sum(edge_0_colors**2)) < 10
    if np.any(edge_0_black == True):
        n_iter += 2

    var_fg = np.where(img_bin == 1)
    colors = cropped_region[:,:,:3][var_fg[0], var_fg[1]]
    foreground_color = np.mean(colors, axis=0)

    colors = {'background': background_color,
              'foreground': foreground_color
    }

    return colors, bg_color, int(round(n_iter*kernel))


def text_removal(input_dir, output_dir, config_path="/content/LAMA/configs/prediction/default.yaml", **kw_args):
    ## processing start
    image_filename_list = glob(input_dir+'/**/**/*.*')
    images_path = [os.path.join(input_dir, file_path) for file_path in image_filename_list]
    print(f"Total number of images: {len(images_path)}")

    ## setting up model params
    with open(config_path, 'r') as f:
        predict_config = OmegaConf.create(yaml.safe_load(f))
    model_path = os.path.join(os.getcwd(), 'big-lama')
    predict_config.model.path = model_path
    predict_config.indir = os.path.dirname(images_path[0])
    predict_config.outdir = output_dir + "/Inpainted"
    predict_config.refiner.gpu_ids = '0'

    train_config_path = os.path.join(predict_config.model.path, 'config.yaml')
    with open(train_config_path, 'r') as f:
        train_config = OmegaConf.create(yaml.safe_load(f))

        train_config.training_model.predict_only = True
        train_config.visualizer.kind = 'noop'

        out_ext = predict_config.get('out_ext', '.png')

        checkpoint_path = os.path.join(predict_config.model.path,
                                       'models',
                                       predict_config.model.checkpoint)
        model = load_checkpoint(train_config, checkpoint_path, strict=False, map_location='cuda' if torch.cuda.is_available() else 'cpu')
        model.freeze()
        if not predict_config.get('refine', False):
            model.to(device)

    ## img processing units
    print(predict_config.dataset) # {'kind': 'default', 'img_suffix': '.png', 'pad_out_to_modulo': 8}
    transform = get_transforms(transform_variant=predict_config.dataset.kind, out_size=512)
    #mask_generator = get_mask_generator(kind=None, kwargs=None) # None train_config.model.mask_schedule
    # print(mask_generator) ## saicinpainting.training.data.masks.MixedMaskGenerator object

    for img_path, img_name in tqdm(zip(images_path, image_filename_list)):
        iteration = 0
        if img_name == img_path:
            img_name = os.path.basename(img_name)
        if 'avif' in img_name:
            img = Image.open(img_path)
            img_path = img_path.replace('avif', 'png')
            img.save(img_path)
            img_name = img_name.replace('avif', 'png')
            del img
        print(img_path, img_name)
        # text detection
        det_results = reader_EF.readtext(img_path, **kw_args)
        img = cv2.imread(img_path)
        img_ = img.copy()
        row, col, channel = img.shape
        img_mask = np.zeros((row, col), dtype=np.uint8)
        radius = 0

        for (bbox, text, prob) in det_results:
            startPoint, endPoint = np.array(bbox[0], dtype=np.int64), np.array(bbox[2], dtype=np.int64)
            points = reduce(add, bbox)
            xy1 = (max(int(points[0]) - radius, 0), max(int(points[1]) - radius, 0))
            xy2 = (min(int(points[2]) + radius, col-1), max(int(points[3]) - radius, 0))
            xy3 = (min(int(points[4]) + radius, col-1), min(int(points[5]) + radius, row-1))
            xy4 = (max(int(points[6]) - radius, 0), min(int(points[7]) + radius, row-1))
            points = np.array([xy1, xy2, xy3, xy4])

            # draw mask on img
            img_mask = drawMask(points, img_mask)

        #img_mask = np.stack((img_mask,)*3, axis=-1)
        #mask = (img_mask[:,:,0]==1)*(img_mask[:,:,1]==0)*(img_mask[:,:,2]==0)

        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(img_mask, cv2.COLOR_BGR2RGB))
        #plt.title(f"Mask of {img_name.split('.')[0]}")
        #plt.show()

        mask = cv2.bitwise_not(img_mask) == 0 # reversed mask
        plt.imsave(os.path.join(output_dir, 'data_for_prediction', f"{os.path.dirname(img_path).split('/')[-1]}_{img_name.split('.')[1]}_mask.png"), mask, cmap='gray')
        copyfile(img_path, os.path.join(output_dir, 'data_for_prediction', os.path.dirname(img_path).split('/')[-1]+'_'+img_name))

        # masked image
        img_inpaint = np.array((1-mask.reshape(mask.shape[0], mask.shape[1], -1))*img[:,:,:3]).astype(np.uint8)
        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(img_inpaint, cv2.COLOR_BGR2RGB))
        #plt.title(f"Masked {img_name.split('.')[0]}")
        #plt.show()
        plt.imsave(os.path.join(output_dir, 'Masked', img_name), cv2.cvtColor(img_inpaint, cv2.COLOR_BGR2RGB))

        # construct in model input format dataset = InpaintingEvaluationDataset(indir, **kwargs)
        predict_config.dataset.img_suffix = img_path.split('.')[-1] # TODO

        img = np.array(Image.open(img_path).convert('RGB')) # default trans
        img = np.transpose(img, (2, 0, 1)) # mode = 'RGB'
        img = img.astype('float32') / 255
        img_mask = img_mask.astype('float32') / 255  # mode = 'L'
        img_batch = [{'image': img, 'mask': img_mask[None, ...], 'unpad_to_size': (row, col)}]

        img_batch = default_collate([img_batch[0]])
        img_batch = move_to_device(img_batch, device)

        cur_res = refine_predict(img_batch, model, **predict_config.refiner)
        cur_res = cur_res[0].permute(1,2,0).detach().cpu().numpy()

        cur_res = np.clip(cur_res * 255, 0, 255).astype('uint8')
        cur_res = cv2.cvtColor(cur_res, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(output_dir, 'Inpainted', os.path.dirname(img_path).split('/')[-1]+'_'+img_name), cur_res)

        res_tile = hconcat_resize_min([img_, img_inpaint, cur_res])
        plt.imsave(os.path.join(output_dir, 'tiled', os.path.dirname(img_path).split('/')[-1]+'_'+img_name), cv2.cvtColor(res_tile, cv2.COLOR_BGR2RGB))
        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(cur_res, cv2.COLOR_BGR2RGB))
        #plt.title(f"Masked {img_name.split('.')[0]}")
        #plt.show()


def text_style_recog_removal(input_dir, output_dit, max_radius=10, lama_config="/content/LAMA/configs/prediction/default.yaml", **kw_args):
    ## processing start
    image_filename_list = glob(input_dir+'/**/**/*.*')[:20]
    images_path = [os.path.join(input_dir, file_path) for file_path in image_filename_list]
    print(f"Total number of images: {len(images_path)}")

    ## setting up LAMA params
    with open(lama_config, 'r') as f:
        predict_config = OmegaConf.create(yaml.safe_load(f))
    model_path = os.path.join(os.getcwd(), 'big-lama')
    predict_config.model.path = model_path
    predict_config.indir = os.path.dirname(images_path[0])
    predict_config.outdir = output_dir + "/Inpainted"
    predict_config.refiner.gpu_ids = '0'

    train_config_path = os.path.join(predict_config.model.path, 'config.yaml')
    with open(train_config_path, 'r') as f:
        train_config = OmegaConf.create(yaml.safe_load(f))

        train_config.training_model.predict_only = True
        train_config.visualizer.kind = 'noop'

        out_ext = predict_config.get('out_ext', '.png')

        checkpoint_path = os.path.join(predict_config.model.path,
                                       'models',
                                       predict_config.model.checkpoint)
        model = load_checkpoint(train_config,
                                checkpoint_path,
                                strict=False,
                                map_location='cuda' if torch.cuda.is_available() else 'cpu')
        model.freeze()
        if not predict_config.get('refine', False):
            model.to(device)

    ## img processing units
    print(predict_config.dataset) # {'kind': 'default', 'img_suffix': '.png', 'pad_out_to_modulo': 8

    for img_path, img_name in tqdm(zip(images_path, image_filename_list)):
        #if 'SKINCEUTICALS-48' not in img_name:
        #    continue
        if img_name == img_path:
            img_name = os.path.basename(img_name)
        if 'avif' in img_name:
            img = Image.open(img_path)
            img_path = img_path.replace('avif', 'png')
            img.save(img_path)
            img_name = img_name.replace('avif', 'png')
            del img
        print(img_path, img_name)
        # text detection
        det_results = reader_EF.readtext(img_path, **kw_args)
        #rec_results = reader_EF.recognize(img_path, det_results)

        img = cv2.imread(img_path)
        img_ = img.copy()
        row, col, channel = img.shape
        img_mask = np.zeros((row, col), dtype=np.uint8)

        for (bbox, text, prob) in det_results:
        #for results in det_results:
        #    bbox, text = results
            points = reduce(add, bbox)

            # text color style
            colors, is_bg_color, edge_wid = style_detect(img, points)
            print(img_name, text, colors)


            if colors is None:
                radius = 0
            elif is_bg_color:
                radius = min(max_radius, edge_wid)
            else:
                radius = 5
            xy1 = (max(int(points[0]) - radius, 0), max(int(points[1]) - radius, 0))
            xy2 = (min(int(points[2]) + radius, col-1), max(int(points[3]) - radius, 0))
            xy3 = (min(int(points[4]) + radius, col-1), min(int(points[5]) + radius, row-1))
            xy4 = (max(int(points[6]) - radius, 0), min(int(points[7]) + radius, row-1))
            points = np.array([xy1, xy2, xy3, xy4])
            # draw mask on img
            img_mask = drawMask(points, img_mask)

        #img_mask = np.stack((img_mask,)*3, axis=-1)
        #mask = (img_mask[:,:,0]==1)*(img_mask[:,:,1]==0)*(img_mask[:,:,2]==0)

        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(img_mask, cv2.COLOR_BGR2RGB))
        #plt.title(f"Mask of {img_name.split('.')[0]}")
        #plt.show()

        mask = cv2.bitwise_not(img_mask) == 0 # reversed mask
        plt.imsave(os.path.join(output_dir, 'data_for_prediction', f"{os.path.dirname(img_path).split('/')[-1]}_{img_name.split('.')[1]}_mask.png"), mask, cmap='gray')
        copyfile(img_path, os.path.join(output_dir, 'data_for_prediction', os.path.dirname(img_path).split('/')[-1]+'_'+img_name))

        # masked image
        img_inpaint = np.array((1-mask.reshape(mask.shape[0], mask.shape[1], -1))*img[:,:,:3]).astype(np.uint8)
        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(img_inpaint, cv2.COLOR_BGR2RGB))
        #plt.title(f"Masked {img_name.split('.')[0]}")
        #plt.show()
        plt.imsave(os.path.join(output_dir, 'Masked', img_name), cv2.cvtColor(img_inpaint, cv2.COLOR_BGR2RGB))

        # construct in model input format dataset = InpaintingEvaluationDataset(indir, **kwargs)
        predict_config.dataset.img_suffix = img_path.split('.')[-1] # TODO

        img = np.array(Image.open(img_path).convert('RGB')) # default trans
        img = np.transpose(img, (2, 0, 1)) # mode = 'RGB'
        img = img.astype('float32') / 255
        img_mask = img_mask.astype('float32') / 255  # mode = 'L'
        img_batch = [{'image': img, 'mask': img_mask[None, ...], 'unpad_to_size': (row, col)}]

        img_batch = default_collate([img_batch[0]])
        img_batch = move_to_device(img_batch, device)

        cur_res = refine_predict(img_batch, model, **predict_config.refiner)
        cur_res = cur_res[0].permute(1,2,0).detach().cpu().numpy()

        cur_res = np.clip(cur_res * 255, 0, 255).astype('uint8')
        cur_res = cv2.cvtColor(cur_res, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(output_dir, 'Inpainted', os.path.dirname(img_path).split('/')[-1]+'_'+img_name), cur_res)

        res_tile = hconcat_resize_min([img_, img_inpaint, cur_res])
        plt.imsave(os.path.join(output_dir, 'tiled', os.path.dirname(img_path).split('/')[-1]+'_'+img_name), cv2.cvtColor(res_tile, cv2.COLOR_BGR2RGB))
        #plt.axis("on")
        #plt.imshow(cv2.cvtColor(cur_res, cv2.COLOR_BGR2RGB))
        #plt.title(f"Masked {img_name.split('.')[0]}")
        #plt.show()


In [5]:
max_radius = 40
input_dir = "/content/gdrive/MyDrive/EasyOCR/Tests" #['batch1', 'batch2', 'batch3', 'batch4']
output_dir = "/content/gdrive/MyDrive/EasyOCR/output/text_detection_products/"
!mkdir -p $output_dir/Inpainted # refinement results
!mkdir -p $output_dir/Masked # masked image
!mkdir -p $output_dir/data_for_prediction # orig_img & mask
!mkdir -p $output_dir/tiled


reader_args = {
    #"paragraph": True, #Combine result into paragraph
    #"detail": 1,
    "rotation_info": [90, 180, 270], # Allow EasyOCR to rotate each text box and return the one with the best confident score. [90, 180 ,270] for all possible text orientations.
    "text_threshold": 0.6, # Text confidence threshold
    "slope_ths": 0.1, # Maximum slope (delta y/delta x) to considered merging. Low value means tiled boxes will not be merged.
    "ycenter_ths": 0.5, # Maximum shift in y direction. Boxes with different level should not be merged.
    "height_ths": 2, # Maximum different in box height. Boxes with very different text size should not be merged.
    "width_ths": 2, # Maximum horizontal distance to merge boxes.
    "add_margin": 0.3, # Extend bounding boxes in all direction by certain value.
    #"x_ths": 2, # Maximum horizontal distance to merge text boxes when paragraph=True.
    #"y_ths": 2, # Maximum verticall distance to merge text boxes when paragraph=True.
    "canvas_size": 4096, # Maximum image size. Image bigger than this value will be resized down.
    "min_size": 10, # Filter text box smaller than minimum value in pixel
    "allowlist": None, # '0123456789'
}


from demo import text_style_recog_removal, text_removal
# run the script
#text_removal(input_dir, output_dir, **reader_args)
text_style_recog_removal(input_dir, output_dir, max_radius=max_radius, **reader_args)

Detectron v2 is not installed
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete recognizer char  !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ªÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþÿĀāĂăĄąĆćČčĎďĐđĒēĖėĘęĚěĞğĨĩĪīĮįİıĶķĹĺĻļĽľŁłŃńŅņŇňŒœŔŕŘřŚśŞşŠšŤťŨũŪūŮůŲųŸŹźŻżŽžƏƠơƯưȘșȚțə̇ḌḍḶḷṀṁṂṃṄṅṆṇṬṭẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ€, <easyocr.utils.CTCLabelConverter object at 0x7e12256d00d0>, 352
Total number of images: 20


/content/LAMA/saicinpainting/training/trainers/__init__.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(path, map_location=map_location)


{'kind': 'default', 'img_suffix': '.png', 'pad_out_to_modulo': 8}


0it [00:00, ?it/s]

...


  0%|          | 0/15 [00:00<?, ?it/s]
                                      
  0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0238:   0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0238:   7%|▋         | 1/15 [00:00<00:03,  3.52it/s]
Refining scale 2 using scale 1 ...current loss: 0.0221:   7%|▋         | 1/15 [00:00<00:03,  3.52it/s]
Refining scale 2 using scale 1 ...current loss: 0.0221:  13%|█▎        | 2/15 [00:00<00:04,  3.06it/s]
Refining scale 2 using scale 1 ...current loss: 0.0164:  13%|█▎        | 2/15 [00:00<00:04,  3.06it/s]
Refining scale 2 using scale 1 ...current loss: 0.0164:  20%|██        | 3/15 [00:01<00:04,  2.93it/s]
Refining scale 2 using scale 1 ...current loss: 0.0153:  20%|██        | 3/15 [00:01<00:04,  2.93it/s]
Refining scale 2 using scale 1 ...current loss: 0.0153:  27%|██▋       | 4/15 [00:01<00:03,  2.87it/s]
Refining scale 2 using scale 1 ...current loss: 0.0135:  27%|██▋  

/content/gdrive/MyDrive/EasyOCR/Tests/batch1/data_instance_SKC001/SKINCEUTICALS-44.png SKINCEUTICALS-44.png
(4426849,)
one_sum: 19548, zero_sum: 1805, redundance 15638.400000000001
SKINCEUTICALS-44.png 50 {'background': array([183.96864129, 211.2859116 , 221.35768365]), 'foreground': array([118.7700831 , 150.46648199, 163.6565097 ])}
(4377522,)
one_sum: 47296, zero_sum: 23384, redundance 37836.8
SKINCEUTICALS-44.png 1 {'background': array([195.12428112, 196.94261671, 204.29287889]), 'foreground': array([119.14381628, 117.28994184, 127.19355115])}
(4443785,)
one_sum: 224, zero_sum: 4193, redundance 179.20000000000002
SKINCEUTICALS-44.png NIXS {'background': array([127.50655855, 127.50655855, 127.50655855]), 'foreground': array([161.46875, 161.46875, 161.46875])}
(4437752,)
one_sum: 8522, zero_sum: 1928, redundance 6817.6
SKINCEUTICALS-44.png 1 {'background': array([184.88476883, 184.8835954 , 184.87866698]), 'foreground': array([102.21939834, 102.22147303, 102.16441909])}
(4438461,)
one


  0%|          | 0/15 [00:00<?, ?it/s]
                                      
  0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0431:   0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0431:   7%|▋         | 1/15 [00:00<00:10,  1.36it/s]
Refining scale 2 using scale 1 ...current loss: 0.0300:   7%|▋         | 1/15 [00:01<00:10,  1.36it/s]
Refining scale 2 using scale 1 ...current loss: 0.0300:  13%|█▎        | 2/15 [00:01<00:10,  1.27it/s]
Refining scale 2 using scale 1 ...current loss: 0.0255:  13%|█▎        | 2/15 [00:02<00:10,  1.27it/s]
Refining scale 2 using scale 1 ...current loss: 0.0255:  20%|██        | 3/15 [00:02<00:09,  1.24it/s]
Refining scale 2 using scale 1 ...current loss: 0.0217:  20%|██        | 3/15 [00:02<00:09,  1.24it/s]
Refining scale 2 using scale 1 ...current loss: 0.0217:  27%|██▋       | 4/15 [00:03<00:08,  1.23it/s]
Refining scale 2 using scale 1 ...current loss: 0.0207:  27%|██▋  

/content/gdrive/MyDrive/EasyOCR/Tests/batch1/data_instance_SKC001/SKINCEUTICALS-53.png SKINCEUTICALS-53.png
(1610741,)
one_sum: 2975, zero_sum: 6588, redundance 2380.0
SKINCEUTICALS-53.png 50+ {'background': array([96.15816636, 88.25880389, 90.50713418]), 'foreground': array([100.18084034, 157.74655462, 191.57042017])}
(1612678,)
one_sum: 6850, zero_sum: 776, redundance 5480.0
SKINCEUTICALS-53.png SKINCEUTICALS {'background': array([184.33270073, 180.25167883, 185.31941606]), 'foreground': array([72.69716495, 70.58376289, 94.97164948])}
(1615981,)
one_sum: 3796, zero_sum: 527, redundance 3036.8
SKINCEUTICALS-53.png ADVANCED {'background': array([178.96127503, 173.42992624, 175.9662803 ]), 'foreground': array([103.51802657,  64.62998102,  40.4573055 ])}
(1614809,)
one_sum: 4796, zero_sum: 699, redundance 3836.8
SKINCEUTICALS-53.png BRIGHTENING {'background': array([183.55337781, 178.75708924, 182.13386155]), 'foreground': array([103.53934192,  67.09728183,  43.        ])}
(1614727,)
one


  0%|          | 0/15 [00:00<?, ?it/s]
                                      
  0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0212:   0%|          | 0/15 [00:00<?, ?it/s]
Refining scale 2 using scale 1 ...current loss: 0.0212:   7%|▋         | 1/15 [00:00<00:09,  1.41it/s]
Refining scale 2 using scale 1 ...current loss: 0.0265:   7%|▋         | 1/15 [00:01<00:09,  1.41it/s]
Refining scale 2 using scale 1 ...current loss: 0.0265:  13%|█▎        | 2/15 [00:01<00:09,  1.43it/s]
Refining scale 2 using scale 1 ...current loss: 0.0141:  13%|█▎        | 2/15 [00:01<00:09,  1.43it/s]
Refining scale 2 using scale 1 ...current loss: 0.0141:  20%|██        | 3/15 [00:02<00:08,  1.42it/s]
Refining scale 2 using scale 1 ...current loss: 0.0132:  20%|██        | 3/15 [00:02<00:08,  1.42it/s]
Refining scale 2 using scale 1 ...current loss: 0.0132:  27%|██▋       | 4/15 [00:02<00:07,  1.42it/s]
Refining scale 2 using scale 1 ...current loss: 0.0125:  27%|██▋  

In [ ]:
torch.cuda.empty_cache()